In [1]:
## Los datos son leidos del sistema HDFS de Hadoop.
## Los resultdos son guardados en una carpeta del sistema Hadoop.
## El script se almacena en un archivo en el disco duro, para su uso posterior.

In [2]:
## Copia Archivos a sistema HDFS
##
## Se usan un directorio temporal en el HDFS. La siguiente
## instrucción muestra el contenido del dicho directorio
##
!hdfs dfs -ls /tmp

Found 2 items
drwxrwx---   - root supergroup          0 2019-11-27 20:05 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2019-11-27 20:10 /tmp/hive


In [3]:
##
## Crea la carpeta wordcount en el hdfs
##
!hdfs dfs -mkdir /tmp/wordcount

In [4]:
##
## Verifica la creación de la carpeta
##
!hdfs dfs -ls /tmp/

Found 3 items
drwxrwx---   - root supergroup          0 2019-11-27 20:05 /tmp/hadoop-yarn
drwxrwxrwx   - root supergroup          0 2019-11-27 20:10 /tmp/hive
drwxr-xr-x   - root supergroup          0 2019-11-27 20:21 /tmp/wordcount


In [5]:
##
## Copia los archvios del directorio local wordcount/
## al directorio /tmp/wordcount/ en el hdfs
##
!hdfs dfs -copyFromLocal wordcount/*  /tmp/wordcount/

In [6]:
##
## Verifica que los archivos esten copiados
## en el hdfs
##
!hdfs dfs -ls /tmp/wordcount

Found 3 items
-rw-r--r--   1 root supergroup       1082 2019-11-27 20:21 /tmp/wordcount/text0.txt
-rw-r--r--   1 root supergroup        349 2019-11-27 20:21 /tmp/wordcount/text1.txt
-rw-r--r--   1 root supergroup        435 2019-11-27 20:21 /tmp/wordcount/text2.txt


In [7]:
####################################################################################################################
## Generación del script y ajuste del código

In [8]:
%%writefile wordcount.hql

DROP TABLE IF EXISTS docs;
DROP TABLE IF EXISTS word_counts;

CREATE TABLE docs (line STRING);

LOAD DATA INPATH "/tmp/wordcount/" OVERWRITE INTO TABLE docs;

CREATE TABLE word_counts
AS
    SELECT word, count(1) AS count
    FROM
        (SELECT explode(split(line, '\\s')) AS word FROM docs) w
GROUP BY
    word
ORDER BY
    word;

INSERT OVERWRITE DIRECTORY '/tmp/output'
ROW FORMAT DELIMITED FIELDS TERMINATED BY ','
SELECT * FROM word_counts;


Writing wordcount.hql


In [9]:
## Ejecucion
!hive -S -e 'source wordcount.hql'

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/usr/local/hive/lib/log4j-slf4j-impl-2.6.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/usr/local/hadoop/share/hadoop/common/lib/slf4j-log4j12-1.7.10.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


In [10]:
## Donde -S indica que Hive se ejecute en modo silencioso; -e que se ejecute la expresión source wordcount.hql. Lo anterior es equivalente a abrir Hive y luego ejecutar:
##      hive> source 'wordcount.hql'

In [11]:
## Visualización de los resultados

In [12]:
## Se lista el contenido del archivo.
!hdfs dfs -ls /tmp/output

Found 1 items
-rwxrwxrwx   1 root supergroup       1652 2019-11-27 20:24 /tmp/output/000000_0


In [13]:
## se visualiza la cabecera del archivo.
!hdfs dfs -cat /tmp/output/000000_0 | head

,1
(DA),1
(see,1
Analytics,2
Analytics,,1
Big,1
Data,3
Especially,1
Organizations,1
Since,1


In [14]:
## Copia de los resultados a la maquina local (vagrant)

In [15]:
!hadoop fs -copyToLocal /tmp/output output
!ls output/*

output/000000_0


In [ ]:
!cat output/000000_0

In [ ]:
## Otra forma de extraer los resultados es: $ hive -S -e 'SELECT * FROM word_counts;' > result.csv

In [ ]:
!rm -rf output wordcount *.hql *.log